In [1]:
from numpy import mean
from numpy import std
from numpy import dstack
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import LSTM
import numpy as np
from tensorflow.keras.metrics import *
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.regularizers import l2

In [2]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.experimental.set_visible_devices(gpus[2], 'GPU')
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

4 Physical GPUs, 1 Logical GPU


In [3]:
s25 = np.load('s25.npy')
h25 = np.load('h25.npy')

In [4]:
s25_labels = np.array([0 for _ in range(0,len(s25))])
h25_labels = np.array([1 for _ in range(0,len(h25))])

In [5]:
X = np.append(s25,h25,axis=0)

In [6]:
y = np.append(s25_labels,h25_labels,axis=0)

In [7]:
X.shape

(1142, 6250, 19)

In [8]:
def evaluate_model(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 1, 50, 128
    n_features = trainX.shape[2]
    n_steps, n_length = 10, 625
    trainX = trainX.reshape((trainX.shape[0], n_steps, n_length, n_features))
    testX = testX.reshape((testX.shape[0], n_steps, n_length, n_features))
    model = Sequential()
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu'),
    input_shape=(None,n_length,n_features)))
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu')))
    model.add(TimeDistributed(Dropout(0.5)))
    model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
    model.add(TimeDistributed(Flatten()))
    model.add(LSTM(100))
    model.add(Dropout(0.5))
    model.add(Dense(100, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(1, activation= 'sigmoid' ))
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=['accuracy',Precision(),Recall(),AUC()])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy,precision,recall,auc = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy,precision,recall,auc

In [9]:
def summarize_results(accuracies,precisions,recalls,aucs):
    m, s = mean(accuracies), std(accuracies)
    print( ' Accuracy: %.3f%% (+/-%.3f) ' % (m, s))
    m, s = mean(precisions), std(precisions)
    print( ' Precision: %.3f%% (+/-%.3f) ' % (m, s))
    m, s = mean(recalls), std(recalls)
    print( ' Recall: %.3f%% (+/-%.3f) ' % (m, s))
    m, s = mean(aucs), std(aucs)
    print( ' AUC: %.3f%% (+/-%.3f) ' % (m, s))

In [10]:
def run_experiment(X,y,repeats=5):
    # load data
    trainX, testX,trainy, testy = train_test_split(X, y, test_size=0.20, random_state=42)
    mean = trainX.mean(axis=0)
    trainX -= mean
    std = trainX.std(axis=0)
    trainX /= std
    testX -= mean
    testX /= std
    # repeat experiment
    accuracies = list()
    precisions = list()
    recalls = list()
    aucs = list()
    for r in range(repeats):
        accuracy,precision,recall,auc = evaluate_model(trainX, trainy, testX, testy)
        accuracy = accuracy * 100.0
        precision = precision * 100.0
        recall = recall * 100.0
        auc = auc * 100.0
        accuracies.append(accuracy)
        precisions.append(precision)
        recalls.append(recall)
        aucs.append(auc)
    # summarize results
    summarize_results(accuracies,precisions,recalls,aucs)

In [11]:
run_experiment(X,y)

Epoch 1/50
8/8 [==============================] - 1s 72ms/step - loss: 0.7162 - accuracy: 0.5356 - precision: 0.5054 - recall: 0.4369 - auc: 0.5444
Epoch 2/50
8/8 [==============================] - 1s 74ms/step - loss: 0.6329 - accuracy: 0.6418 - precision: 0.6135 - recall: 0.6379 - auc: 0.7001
Epoch 3/50
8/8 [==============================] - 1s 74ms/step - loss: 0.5173 - accuracy: 0.7766 - precision: 0.7171 - recall: 0.8645 - auc: 0.8639
Epoch 4/50
8/8 [==============================] - 1s 79ms/step - loss: 0.3614 - accuracy: 0.8762 - precision: 0.8706 - recall: 0.8645 - auc: 0.9409
Epoch 5/50
8/8 [==============================] - 0s 60ms/step - loss: 0.2541 - accuracy: 0.9211 - precision: 0.9300 - recall: 0.8995 - auc: 0.9718
Epoch 6/50
8/8 [==============================] - 1s 70ms/step - loss: 0.2012 - accuracy: 0.9299 - precision: 0.9333 - recall: 0.9159 - auc: 0.9760
Epoch 7/50
8/8 [==============================] - 1s 71ms/step - loss: 0.1476 - accuracy: 0.9529 - precision: 0.

8/8 [==============================] - 0s 57ms/step - loss: 0.3425 - accuracy: 0.8927 - precision_1: 0.8947 - recall_1: 0.8738 - auc_1: 0.9554
Epoch 6/50
8/8 [==============================] - 1s 69ms/step - loss: 0.2557 - accuracy: 0.9179 - precision_1: 0.9133 - recall_1: 0.9112 - auc_1: 0.9725
Epoch 7/50
8/8 [==============================] - 1s 69ms/step - loss: 0.1842 - accuracy: 0.9551 - precision_1: 0.9574 - recall_1: 0.9463 - auc_1: 0.9857
Epoch 8/50
8/8 [==============================] - 1s 64ms/step - loss: 0.1742 - accuracy: 0.9441 - precision_1: 0.9333 - recall_1: 0.9486 - auc_1: 0.9812
Epoch 9/50
8/8 [==============================] - 1s 66ms/step - loss: 0.1354 - accuracy: 0.9617 - precision_1: 0.9538 - recall_1: 0.9650 - auc_1: 0.9886
Epoch 10/50
8/8 [==============================] - 1s 68ms/step - loss: 0.2065 - accuracy: 0.9211 - precision_1: 0.9258 - recall_1: 0.9042 - auc_1: 0.9735
Epoch 11/50
8/8 [==============================] - 1s 70ms/step - loss: 0.1167 - accur

8/8 [==============================] - 1s 63ms/step - loss: 0.0862 - accuracy: 0.9792 - precision_2: 0.9789 - recall_2: 0.9766 - auc_2: 0.9933
Epoch 9/50
8/8 [==============================] - 1s 76ms/step - loss: 0.0728 - accuracy: 0.9770 - precision_2: 0.9744 - recall_2: 0.9766 - auc_2: 0.9968
Epoch 10/50
8/8 [==============================] - 1s 69ms/step - loss: 0.0524 - accuracy: 0.9880 - precision_2: 0.9860 - recall_2: 0.9883 - auc_2: 0.9964
Epoch 11/50
8/8 [==============================] - 1s 75ms/step - loss: 0.0406 - accuracy: 0.9912 - precision_2: 0.9907 - recall_2: 0.9907 - auc_2: 0.9976
Epoch 12/50
8/8 [==============================] - 1s 76ms/step - loss: 0.0300 - accuracy: 0.9945 - precision_2: 0.9976 - recall_2: 0.9907 - auc_2: 0.9982
Epoch 13/50
8/8 [==============================] - 1s 68ms/step - loss: 0.0250 - accuracy: 0.9934 - precision_2: 0.9953 - recall_2: 0.9907 - auc_2: 0.9993
Epoch 14/50
8/8 [==============================] - 0s 57ms/step - loss: 0.0131 - ac

8/8 [==============================] - 1s 74ms/step - loss: 0.1453 - accuracy: 0.9584 - precision_3: 0.9577 - recall_3: 0.9533 - auc_3: 0.9905
Epoch 11/50
8/8 [==============================] - 1s 75ms/step - loss: 0.1010 - accuracy: 0.9715 - precision_3: 0.9674 - recall_3: 0.9720 - auc_3: 0.9961
Epoch 12/50
8/8 [==============================] - 0s 62ms/step - loss: 0.0811 - accuracy: 0.9803 - precision_3: 0.9858 - recall_3: 0.9720 - auc_3: 0.9961
Epoch 13/50
8/8 [==============================] - 1s 65ms/step - loss: 0.0567 - accuracy: 0.9923 - precision_3: 0.9930 - recall_3: 0.9907 - auc_3: 0.9992
Epoch 14/50
8/8 [==============================] - 1s 69ms/step - loss: 0.0483 - accuracy: 0.9901 - precision_3: 0.9883 - recall_3: 0.9907 - auc_3: 0.9993
Epoch 15/50
8/8 [==============================] - 1s 63ms/step - loss: 0.0489 - accuracy: 0.9890 - precision_3: 0.9860 - recall_3: 0.9907 - auc_3: 0.9993
Epoch 16/50
8/8 [==============================] - 1s 66ms/step - loss: 0.0387 - a

8/8 [==============================] - 1s 84ms/step - loss: 0.0552 - accuracy: 0.9901 - precision_4: 0.9883 - recall_4: 0.9907 - auc_4: 0.9989
Epoch 13/50
8/8 [==============================] - 1s 74ms/step - loss: 0.0502 - accuracy: 0.9890 - precision_4: 0.9883 - recall_4: 0.9883 - auc_4: 0.9986
Epoch 14/50
8/8 [==============================] - 1s 71ms/step - loss: 0.0404 - accuracy: 0.9934 - precision_4: 0.9953 - recall_4: 0.9907 - auc_4: 0.9992
Epoch 15/50
8/8 [==============================] - 1s 73ms/step - loss: 0.0339 - accuracy: 0.9912 - precision_4: 0.9907 - recall_4: 0.9907 - auc_4: 0.9995
Epoch 16/50
8/8 [==============================] - 1s 90ms/step - loss: 0.0179 - accuracy: 0.9978 - precision_4: 0.9977 - recall_4: 0.9977 - auc_4: 1.0000
Epoch 17/50
8/8 [==============================] - 1s 73ms/step - loss: 0.0202 - accuracy: 0.9967 - precision_4: 1.0000 - recall_4: 0.9930 - auc_4: 0.9998
Epoch 18/50
8/8 [==============================] - 1s 63ms/step - loss: 0.0140 - a

In [12]:
def evaluate_model_2(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 1, 50, 128
    n_features = trainX.shape[2]
    n_steps, n_length = 10, 625
    trainX = trainX.reshape((trainX.shape[0], n_steps, n_length, n_features))
    testX = testX.reshape((testX.shape[0], n_steps, n_length, n_features))
    model = Sequential()
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3),input_shape=(None,n_length,n_features)))
    model.add(LeakyReLU())
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3)))
    model.add(LeakyReLU())
    model.add(TimeDistributed(Dropout(0.5)))
    model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
    model.add(TimeDistributed(Flatten()))
    model.add(LSTM(100))
    model.add(Dropout(0.5))
    model.add(Dense(100))
    model.add(LeakyReLU())
    model.add(Dropout(0.25))
    model.add(Dense(1, activation= 'sigmoid' ))
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=['accuracy',Precision(),Recall(),AUC()])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy,precision,recall,auc = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy,precision,recall,auc

def run_experiment_2(X,y,repeats=5):
    # load data
    trainX, testX,trainy, testy = train_test_split(X, y, test_size=0.20, random_state=42)
    mean = trainX.mean(axis=0)
    trainX -= mean
    std = trainX.std(axis=0)
    trainX /= std
    testX -= mean
    testX /= std
    # repeat experiment
    accuracies = list()
    precisions = list()
    recalls = list()
    aucs = list()
    for r in range(repeats):
        accuracy,precision,recall,auc = evaluate_model_2(trainX, trainy, testX, testy)
        accuracy = accuracy * 100.0
        precision = precision * 100.0
        recall = recall * 100.0
        auc = auc * 100.0
        accuracies.append(accuracy)
        precisions.append(precision)
        recalls.append(recall)
        aucs.append(auc)
    # summarize results
    summarize_results(accuracies,precisions,recalls,aucs)
    
run_experiment_2(X,y)

Epoch 1/50
8/8 [==============================] - 1s 64ms/step - loss: 0.8198 - accuracy: 0.5093 - precision_5: 0.4788 - recall_5: 0.5280 - auc_5: 0.4984
Epoch 2/50
8/8 [==============================] - 1s 68ms/step - loss: 0.6906 - accuracy: 0.5520 - precision_5: 0.5222 - recall_5: 0.5210 - auc_5: 0.5789
Epoch 3/50
8/8 [==============================] - 1s 81ms/step - loss: 0.6692 - accuracy: 0.5739 - precision_5: 0.5589 - recall_5: 0.4322 - auc_5: 0.6181
Epoch 4/50
8/8 [==============================] - 1s 63ms/step - loss: 0.5746 - accuracy: 0.7097 - precision_5: 0.6927 - recall_5: 0.6846 - auc_5: 0.7869
Epoch 5/50
8/8 [==============================] - 1s 64ms/step - loss: 0.4906 - accuracy: 0.7820 - precision_5: 0.8053 - recall_5: 0.7056 - auc_5: 0.8791
Epoch 6/50
8/8 [==============================] - 0s 62ms/step - loss: 0.3855 - accuracy: 0.8697 - precision_5: 0.8815 - recall_5: 0.8341 - auc_5: 0.9418
Epoch 7/50
8/8 [==============================] - 1s 67ms/step - loss: 0.270

Epoch 1/50
8/8 [==============================] - 1s 75ms/step - loss: 0.7266 - accuracy: 0.5159 - precision_6: 0.4823 - recall_6: 0.4463 - auc_6: 0.5239
Epoch 2/50
8/8 [==============================] - 1s 69ms/step - loss: 0.6488 - accuracy: 0.6156 - precision_6: 0.5877 - recall_6: 0.6028 - auc_6: 0.6580
Epoch 3/50
8/8 [==============================] - 0s 61ms/step - loss: 0.5406 - accuracy: 0.7514 - precision_6: 0.7072 - recall_6: 0.8014 - auc_6: 0.8306
Epoch 4/50
8/8 [==============================] - 1s 64ms/step - loss: 0.3915 - accuracy: 0.8565 - precision_6: 0.8180 - recall_6: 0.8925 - auc_6: 0.9376
Epoch 5/50
8/8 [==============================] - 0s 60ms/step - loss: 0.2978 - accuracy: 0.8938 - precision_6: 0.8858 - recall_6: 0.8879 - auc_6: 0.9535
Epoch 6/50
8/8 [==============================] - 1s 68ms/step - loss: 0.1916 - accuracy: 0.9398 - precision_6: 0.9368 - recall_6: 0.9346 - auc_6: 0.9821
Epoch 7/50
8/8 [==============================] - 1s 76ms/step - loss: 0.147

Epoch 1/50
8/8 [==============================] - 1s 63ms/step - loss: 0.6702 - accuracy: 0.5893 - precision_7: 0.5681 - recall_7: 0.5164 - auc_7: 0.6240
Epoch 2/50
8/8 [==============================] - 1s 63ms/step - loss: 0.5681 - accuracy: 0.7065 - precision_7: 0.6732 - recall_7: 0.7266 - auc_7: 0.7708
Epoch 3/50
8/8 [==============================] - 1s 77ms/step - loss: 0.4124 - accuracy: 0.8182 - precision_7: 0.7873 - recall_7: 0.8388 - auc_7: 0.9028
Epoch 4/50
8/8 [==============================] - 0s 60ms/step - loss: 0.3127 - accuracy: 0.8751 - precision_7: 0.8428 - recall_7: 0.9019 - auc_7: 0.9449
Epoch 5/50
8/8 [==============================] - 1s 63ms/step - loss: 0.1871 - accuracy: 0.9430 - precision_7: 0.9393 - recall_7: 0.9393 - auc_7: 0.9832
Epoch 6/50
8/8 [==============================] - 0s 61ms/step - loss: 0.1296 - accuracy: 0.9573 - precision_7: 0.9534 - recall_7: 0.9556 - auc_7: 0.9899
Epoch 7/50
8/8 [==============================] - 0s 62ms/step - loss: 0.099

Epoch 1/50
8/8 [==============================] - 1s 78ms/step - loss: 0.7533 - accuracy: 0.4984 - precision_8: 0.4678 - recall_8: 0.5093 - auc_8: 0.5149
Epoch 2/50
8/8 [==============================] - 0s 62ms/step - loss: 0.6591 - accuracy: 0.6199 - precision_8: 0.6080 - recall_8: 0.5327 - auc_8: 0.6491
Epoch 3/50
8/8 [==============================] - 0s 56ms/step - loss: 0.5681 - accuracy: 0.7141 - precision_8: 0.6928 - recall_8: 0.7009 - auc_8: 0.7820
Epoch 4/50
8/8 [==============================] - 1s 68ms/step - loss: 0.4099 - accuracy: 0.8324 - precision_8: 0.8345 - recall_8: 0.8014 - auc_8: 0.9170
Epoch 5/50
8/8 [==============================] - 0s 60ms/step - loss: 0.2747 - accuracy: 0.9069 - precision_8: 0.8820 - recall_8: 0.9252 - auc_8: 0.9635
Epoch 6/50
8/8 [==============================] - 1s 81ms/step - loss: 0.1923 - accuracy: 0.9387 - precision_8: 0.9326 - recall_8: 0.9369 - auc_8: 0.9821
Epoch 7/50
8/8 [==============================] - 0s 61ms/step - loss: 0.134

Epoch 1/50
8/8 [==============================] - 1s 87ms/step - loss: 0.8380 - accuracy: 0.5148 - precision_9: 0.4661 - recall_9: 0.2407 - auc_9: 0.4883
Epoch 2/50
8/8 [==============================] - 1s 76ms/step - loss: 0.7104 - accuracy: 0.5060 - precision_9: 0.4811 - recall_9: 0.6846 - auc_9: 0.5236
Epoch 3/50
8/8 [==============================] - 1s 77ms/step - loss: 0.6791 - accuracy: 0.5608 - precision_9: 0.5271 - recall_9: 0.6145 - auc_9: 0.5914
Epoch 4/50
8/8 [==============================] - 1s 81ms/step - loss: 0.6545 - accuracy: 0.6112 - precision_9: 0.5788 - recall_9: 0.6262 - auc_9: 0.6647
Epoch 5/50
8/8 [==============================] - 1s 76ms/step - loss: 0.5976 - accuracy: 0.7032 - precision_9: 0.7289 - recall_9: 0.5841 - auc_9: 0.7927
Epoch 6/50
8/8 [==============================] - 1s 81ms/step - loss: 0.5034 - accuracy: 0.8061 - precision_9: 0.8329 - recall_9: 0.7336 - auc_9: 0.9095
Epoch 7/50
8/8 [==============================] - 1s 82ms/step - loss: 0.403

 Accuracy: 74.061% (+/-19.978) 
 Precision: 65.837% (+/-27.499) 
 Recall: 58.409% (+/-32.916) 
 AUC: 78.327% (+/-20.994) 


In [13]:
def evaluate_model_3(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 1, 50, 128
    n_features = trainX.shape[2]
    n_steps, n_length = 10, 625
    trainX = trainX.reshape((trainX.shape[0], n_steps, n_length, n_features))
    testX = testX.reshape((testX.shape[0], n_steps, n_length, n_features))
    model = Sequential()
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='selu'),
    input_shape=(None,n_length,n_features)))
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='selu')))
    model.add(TimeDistributed(Dropout(0.5)))
    model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
    model.add(TimeDistributed(Flatten()))
    model.add(LSTM(100))
    model.add(Dropout(0.5))
    model.add(Dense(100, activation='selu'))
    model.add(Dropout(0.25))
    model.add(Dense(1, activation= 'sigmoid' ))
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=['accuracy',Precision(),Recall(),AUC()])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy,precision,recall,auc = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy,precision,recall,auc

def run_experiment_3(X,y,repeats=5):
    # load data
    trainX, testX,trainy, testy = train_test_split(X, y, test_size=0.20, random_state=42)
    mean = trainX.mean(axis=0)
    trainX -= mean
    std = trainX.std(axis=0)
    trainX /= std
    testX -= mean
    testX /= std
    # repeat experiment
    accuracies = list()
    precisions = list()
    recalls = list()
    aucs = list()
    for r in range(repeats):
        accuracy,precision,recall,auc = evaluate_model_3(trainX, trainy, testX, testy)
        accuracy = accuracy * 100.0
        precision = precision * 100.0
        recall = recall * 100.0
        auc = auc * 100.0
        accuracies.append(accuracy)
        precisions.append(precision)
        recalls.append(recall)
        aucs.append(auc)
    # summarize results
    summarize_results(accuracies,precisions,recalls,aucs)
    
run_experiment_3(X,y)

Epoch 1/50
8/8 [==============================] - 1s 75ms/step - loss: 0.8642 - accuracy: 0.5323 - precision_10: 0.5010 - recall_10: 0.5864 - auc_10: 0.5382
Epoch 2/50
8/8 [==============================] - 1s 66ms/step - loss: 0.6801 - accuracy: 0.5641 - precision_10: 0.5421 - recall_10: 0.4509 - auc_10: 0.6133
Epoch 3/50
8/8 [==============================] - 1s 67ms/step - loss: 0.5721 - accuracy: 0.7076 - precision_10: 0.6716 - recall_10: 0.7360 - auc_10: 0.7795
Epoch 4/50
8/8 [==============================] - 1s 70ms/step - loss: 0.4658 - accuracy: 0.7974 - precision_10: 0.7900 - recall_10: 0.7734 - auc_10: 0.8740
Epoch 5/50
8/8 [==============================] - 0s 62ms/step - loss: 0.3541 - accuracy: 0.8554 - precision_10: 0.8491 - recall_10: 0.8411 - auc_10: 0.9294
Epoch 6/50
8/8 [==============================] - 1s 63ms/step - loss: 0.2703 - accuracy: 0.9036 - precision_10: 0.8846 - recall_10: 0.9136 - auc_10: 0.9629
Epoch 7/50
8/8 [==============================] - 0s 57ms/

Epoch 1/50
8/8 [==============================] - 1s 70ms/step - loss: 0.7793 - accuracy: 0.5750 - precision_11: 0.5565 - recall_11: 0.4603 - auc_11: 0.5854
Epoch 2/50
8/8 [==============================] - 0s 58ms/step - loss: 0.6318 - accuracy: 0.6484 - precision_11: 0.6055 - recall_11: 0.7173 - auc_11: 0.7038
Epoch 3/50
8/8 [==============================] - 0s 60ms/step - loss: 0.5240 - accuracy: 0.7349 - precision_11: 0.7204 - recall_11: 0.7103 - auc_11: 0.8184
Epoch 4/50
8/8 [==============================] - 1s 67ms/step - loss: 0.3782 - accuracy: 0.8313 - precision_11: 0.8072 - recall_11: 0.8411 - auc_11: 0.9215
Epoch 5/50
8/8 [==============================] - 1s 65ms/step - loss: 0.2735 - accuracy: 0.8872 - precision_11: 0.8753 - recall_11: 0.8855 - auc_11: 0.9581
Epoch 6/50
8/8 [==============================] - 1s 68ms/step - loss: 0.1755 - accuracy: 0.9332 - precision_11: 0.9318 - recall_11: 0.9252 - auc_11: 0.9823
Epoch 7/50
8/8 [==============================] - 1s 74ms/

Epoch 1/50
8/8 [==============================] - 0s 59ms/step - loss: 0.9311 - accuracy: 0.4896 - precision_12: 0.4632 - recall_12: 0.5584 - auc_12: 0.4991
Epoch 2/50
8/8 [==============================] - 0s 58ms/step - loss: 0.7377 - accuracy: 0.5411 - precision_12: 0.5130 - recall_12: 0.4159 - auc_12: 0.5498
Epoch 3/50
8/8 [==============================] - 0s 58ms/step - loss: 0.6835 - accuracy: 0.5936 - precision_12: 0.5714 - recall_12: 0.5327 - auc_12: 0.6170
Epoch 4/50
8/8 [==============================] - 0s 55ms/step - loss: 0.6329 - accuracy: 0.6331 - precision_12: 0.6274 - recall_12: 0.5350 - auc_12: 0.6903
Epoch 5/50
8/8 [==============================] - 1s 63ms/step - loss: 0.5471 - accuracy: 0.7317 - precision_12: 0.7226 - recall_12: 0.6939 - auc_12: 0.7978
Epoch 6/50
8/8 [==============================] - 0s 53ms/step - loss: 0.4458 - accuracy: 0.8050 - precision_12: 0.7976 - recall_12: 0.7827 - auc_12: 0.8857
Epoch 7/50
8/8 [==============================] - 0s 53ms/

Epoch 1/50
8/8 [==============================] - 0s 55ms/step - loss: 0.6803 - accuracy: 0.5816 - precision_13: 0.5553 - recall_13: 0.5397 - auc_13: 0.6338
Epoch 2/50
8/8 [==============================] - 0s 53ms/step - loss: 0.5228 - accuracy: 0.7426 - precision_13: 0.7292 - recall_13: 0.7173 - auc_13: 0.8165
Epoch 3/50
8/8 [==============================] - 0s 56ms/step - loss: 0.3760 - accuracy: 0.8215 - precision_13: 0.7964 - recall_13: 0.8318 - auc_13: 0.9141
Epoch 4/50
8/8 [==============================] - 0s 55ms/step - loss: 0.2311 - accuracy: 0.9069 - precision_13: 0.9016 - recall_13: 0.8995 - auc_13: 0.9706
Epoch 5/50
8/8 [==============================] - 0s 56ms/step - loss: 0.1282 - accuracy: 0.9562 - precision_13: 0.9597 - recall_13: 0.9463 - auc_13: 0.9922
Epoch 6/50
8/8 [==============================] - 0s 55ms/step - loss: 0.0709 - accuracy: 0.9825 - precision_13: 0.9769 - recall_13: 0.9860 - auc_13: 0.9980
Epoch 7/50
8/8 [==============================] - 0s 53ms/

Epoch 1/50
8/8 [==============================] - 0s 56ms/step - loss: 0.8367 - accuracy: 0.5246 - precision_14: 0.4935 - recall_14: 0.5304 - auc_14: 0.5347
Epoch 2/50
8/8 [==============================] - 0s 54ms/step - loss: 0.6451 - accuracy: 0.6320 - precision_14: 0.5954 - recall_14: 0.6706 - auc_14: 0.6881
Epoch 3/50
8/8 [==============================] - 0s 56ms/step - loss: 0.5360 - accuracy: 0.7327 - precision_14: 0.7738 - recall_14: 0.6075 - auc_14: 0.8165
Epoch 4/50
8/8 [==============================] - 0s 56ms/step - loss: 0.4115 - accuracy: 0.8226 - precision_14: 0.8037 - recall_14: 0.8224 - auc_14: 0.9042
Epoch 5/50
8/8 [==============================] - 0s 56ms/step - loss: 0.2798 - accuracy: 0.9102 - precision_14: 0.8932 - recall_14: 0.9182 - auc_14: 0.9558
Epoch 6/50
8/8 [==============================] - 0s 52ms/step - loss: 0.2148 - accuracy: 0.9113 - precision_14: 0.8934 - recall_14: 0.9206 - auc_14: 0.9750
Epoch 7/50
8/8 [==============================] - 0s 53ms/

 Accuracy: 59.738% (+/-1.472) 
 Precision: 41.140% (+/-6.920) 
 Recall: 8.409% (+/-3.182) 
 AUC: 50.951% (+/-2.379) 
